In [1]:
import pandas as pd
import numpy as np
import json
import time
import atexit
from datetime import datetime
from datetime import date
from tda.client import Client
from tda.auth import easy_client
from tda import auth
from tda.orders import equities
from tda.orders.common import Duration, Session

In [4]:
# Set account variables and webdriver path
account_id = open('/mnt/c/Python/API Keys/TD/TD_ACCOUNT_ID.txt').read()
consumer_key = open('/mnt/c/Python/API Keys/TD/TD_CONSUMER_KEY.txt').read()
redirect_uri = 'http://localhost'
token_path = '/mnt/c/Python/API Keys/TD/ameritrade-credentials.json'
geckodriver_path = '/usr/local/bin/geckodriver'
# Creates Webdriver for Selenium
def make_webdriver():
    # Import selenium here because it's slow to import
    from selenium import webdriver
    driver = webdriver.Firefox(executable_path = geckodriver_path)
    atexit.register(lambda: driver.quit())
    return driver

In [95]:
# Create client object
c = easy_client(consumer_key,
                            redirect_uri,
                            token_path,
                            make_webdriver)

In [113]:
class TDformatter:
    
    class Quote_formatter:
        """
        Returns formatted pandas DataFrame of quotes.
        """
        def __init__(self, response):
            self.data = response.json()

        def to_df(self):
            return pd.DataFrame(self.data).T
    
    
    class History_formatter:
        """
        Returns formatted pandas DataFrame of history. 
        """
        def __init__(self, response):
            self.data = response.json()
            self.candles = self.data['candles']
            self.ticker = self.data['symbol']

        def to_df(self):
            self.dataframe = pd.DataFrame(self.candles)
            self.dataframe['symbol'] = self.ticker
            self.dataframe.set_index('datetime', inplace=True)
#             Converts time since epoch to a datetime object with second accuracy (any more is unneccessary)
#             and sets it as index.
            self.dataframe.index = pd.to_datetime((self.dataframe.index/1000).astype('int64'), unit='s')
            return self.dataframe
        
        
    class Chain_formatter:
        """
        Returns formatted pandas DataFrame of options chains.  Can take call &/or put chains, if given 
        both will return tuple of calls, puts.
        """
        def __init__(self, response):
            self.data = response.json()
#             Check if the options_chain response contains calls, if empty then set flag to false
            if self.data['callExpDateMap'] != {}:
                self.calls = self.data['callExpDateMap'].values()
                self.calls_exists = True
            else:
                self.calls_exists = False
            
#             Check if the options_chain response contains puts, if empty then set flag to false.
            if self.data['putExpDateMap'] != {}:
                self.puts = self.data['putExpDateMap'].values()
                self.puts_exists = True
            else:
                self.puts_exists = False
        
        def to_df(self):
#             Gets today's date for use in index.
            today = datetime.now().strftime('%m-%d-%Y')
            
#             Checks if options_chain response contains calls, if not then skips.
            if self.calls_exists:
                self.calls_list = []
#             **Very inefficient triple for loop, should be revised if possible**
                for i in self.calls:
                    for j in i.values():
                        for k in j:
                            self.calls_list.append(k)
                self.calls_df = pd.DataFrame(self.calls_list)
                self.calls_df.set_index('description', inplace=True)
                self.calls_df['Date'] = today
#                 Removes garbage responses with -999 deltas
                self.calls_df[self.calls_df['delta'] != -999.0]
            
#             Checks if options_chain response contains puts, if not then skips.
            if self.puts_exists:
                self.puts_list = []
#             **Very inefficient triple for loop, should be revised if possible**
                for i in self.puts:
                    for j in i.values():
                        for k in j:
                            self.puts_list.append(k)
                self.puts_df = pd.DataFrame(self.puts_list)
                self.puts_df.set_index('description', inplace=True)
                self.puts_df['Date'] = today
#                 Removes garbage responses with -999 deltas
                self.puts_df[self.puts_df['delta'] != -999.0]
            
#            Determines return values based on if calls &/or puts exists.
            if self.calls_exists and self.puts_exists:
                return self.calls_df, self.puts_df
            elif self.calls_exists and not self.puts_exists:
                return self.calls_df
            elif not self.calls_exists and self.puts_exists:
                return self.puts_df

In [114]:
symbol_list = ['SPY', 'QQQ']

In [117]:
quote_response = c.get_quotes(symbol_list)
TDformatter().Quote_formatter(quote_response).to_df()

assetType assetMainType      cusip assetSubType symbol  \
SPY       ETF        EQUITY  78462F103          ETF    SPY   
QQQ       ETF        EQUITY  46090E103          ETF    QQQ   

                     description bidPrice bidSize bidId askPrice  ...  \
SPY                 SPDR S&P 500   433.77     100     P   433.84  ...   
QQQ  Invesco QQQ Trust, Series 1    359.6    1600     P   359.63  ...   

    regularMarketLastPrice regularMarketLastSize regularMarketNetChange  \
SPY                 434.75                     4                  -1.49   
QQQ                 360.52                  8807                  -2.55   

    regularMarketTradeTimeInLong netPercentChangeInDouble markChangeInDouble  \
SPY                1626393600002                  -0.3416               -2.4   
QQQ                1626379200887                  -0.9475              -3.44   

    markPercentChangeInDouble regularMarketPercentChangeInDouble delayed  \
SPY                   -0.5502                            -0.3416    True   
QQQ                   -0.9475                            -0.7023    True   

    realtimeEntitled  
SPY            False  
QQQ            False  

[2 rows x 49 columns]

In [11]:
history_dict = {}
for symbol in symbol_list:
    history_response = c.get_price_history(symbol = symbol,
                                        period_type = Client.PriceHistory.PeriodType.YEAR,
                                        period = Client.PriceHistory.Period.TWENTY_YEARS,
                                        frequency_type = Client.PriceHistory.FrequencyType.DAILY,
                                        frequency = Client.PriceHistory.Frequency.DAILY)
    history_dict[symbol] = TDformatter.History_formatter(history_response).to_df()

In [115]:
chain_dict = {}
for symbol in symbol_list:
    chain_response = c.get_option_chain(symbol = symbol,
                      contract_type = Client.Options.ContractType.CALL,
                      strike_range = Client.Options.StrikeRange.ALL)
    chain_dict[symbol] = TDformatter().Chain_formatter(chain_response).to_df()